# TP: Transfer Learning for MNIST Classification

Here you will learn how to use transfer learning to adapt a pre-trained CNN for a new classification task. We will use a small but performant network pre-trained on ImageNet (1000 classes) and adapt it to classify MNIST handwritten digits (10 classes).

**Learning objectives:**
- Understand transfer learning concepts
- Load and prepare the MNIST dataset
- Use a pre-trained model and modify its classification head
- Train only the new layers while freezing pre-trained weights
- Evaluate model performance on a validation set

## System setup

Import the required packages and check PyTorch version.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, datasets
import time

device = "cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"{torch.__version__=}")
print(f"Using {device=}")

## Download the MNIST dataset

MNIST is a classic dataset of 28x28 grayscale images of handwritten digits (0-9). It contains:
- 60,000 training images
- 10,000 test images

PyTorch will automatically download the dataset for you.

## Data processing

**Important:** Pre-trained models expect input in a specific format:
- Images must be resized to 224x224 (ImageNet standard)
- Images must have 3 color channels (RGB)
- Images must be normalized with ImageNet statistics

Since MNIST images are:
- 28x28 pixels (too small)
- Grayscale (1 channel instead of 3)

We need to transform them appropriately.

**TODO 1:** Complete the transform pipeline below to:
1. Resize MNIST images to 224x224
2. Convert grayscale to RGB (3 channels)
3. Convert to tensor
4. Normalize with ImageNet statistics

Check the doc of the following transforms: 
- [transforms.Resize](https://docs.pytorch.org/vision/main/generated/torchvision.transforms.Resize.html)
- [transforms.Grayscale](https://docs.pytorch.org/vision/main/generated/torchvision.transforms.Grayscale.html)

In [ ]:
# ImageNet normalization statistics
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# TODO 1: Complete this transform pipeline
mnist_transform = transforms.Compose([
    #TODO
])

**TODO 2:** Load the MNIST dataset for training and testing using `datasets.MNIST`.

Hints:
- Set `train=True` for training set, `train=False` for test set
- Set `download=True` to download the dataset
- Use the `mnist_transform` we defined above

MNIST is so popular, the dataset is already available in torchvision: check the doc for [datasets.MNIST](https://docs.pytorch.org/vision/main/generated/torchvision.datasets.MNIST.html)

In [ ]:
# TODO 2: Load MNIST datasets
train_dataset = {} #TODO
test_dataset = {} #TODO

In [ ]:
# Check dataset sizes
dset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}
print(f"Training set size: {dset_sizes['train']}")
print(f"Test set size: {dset_sizes['test']}")

Let's examine the MNIST class labels:

In [ ]:
mnist_classes = train_dataset.classes
print(f"MNIST classes: {mnist_classes}")
print(f"Number of classes: {len(mnist_classes)}")

**TODO 3:** Create DataLoaders for training and testing.

Hints:
- Use batch_size=64 for training
- Use batch_size=100 for testing
- Shuffle training data but not test data
- Set num_workers=2 (lower than dogs/cats example for CPU training)

Check the doc for [torch.utils.data.DataLoader](https://docs.pytorch.org/docs/stable/data.html)

In [ ]:
# TODO 3: Create DataLoaders
train_loader = [] #TODO
test_loader = [] #TODO

Let's visualize some examples from the training set:

In [ ]:
# Get a batch of training data
inputs, labels = next(iter(train_loader))

print(f"Batch shape: {inputs.shape}")
print(f"Labels: {labels[:8]}")

In [ ]:
def imshow(inp, title=None):
    """Display image from tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = np.clip(std * inp + mean, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.axis('off')

# Make a grid from batch
n_images = 8
out = torchvision.utils.make_grid(inputs[0:n_images])
imshow(out, title=[str(x.item()) for x in labels[0:n_images]])

## Load a pre-trained model: MobileNetV3 

We will use [**MobileNetV3-small**](https://huggingface.co/qualcomm/MobileNet-v3-Small), a small and efficient CNN designed for mobile devices. Despite its small size (~2.5M parameters), it achieves relatively good performance on ImageNet for its size.

Let's load the pre-trained model:

In [ ]:
# Load pre-trained MobileNetV2
model = models.mobilenet_v3_small(weights='DEFAULT')
print(model)

Let's examine the structure of MobileNetV2:
- `features`: Convolutional layers for feature extraction
- `classifier`: Final classification layer(s)

The classifier outputs 1000 classes (ImageNet). We need to modify it for 10 classes (MNIST digits).

In [ ]:
print("\nClassifier structure:")
print(model.classifier)

## Modify the classification head

**Transfer learning strategy:**
1. Freeze all pre-trained weights (no gradient computation)
2. Replace the final layer to output 10 classes instead of 1000
3. Train only the new layer

**TODO 4:** Complete the code below to:
1. Freeze all parameters in the model
2. Replace the final linear layer with a new one for 10 classes

Hints:
- The classifier is a Sequential with Dropout and Linear layers
- The Linear layer is at index 3: `model.classifier[3]`
- The input features to this layer is 1024 (check the print above)

In [ ]:
# TODO 4: Freeze all parameters and replace final layer

# Freeze all parameters
#TODO

# Replace the final layer: Linear(1024, 1000) -> Linear(1280, 10)
#TODO

print("\nModified classifier:")
print(model.classifier)

In [ ]:
# Move model to device (GPU/MPS/CPU)
model = model.to(device)

# Verify: count trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Frozen parameters: {total_params - trainable_params:,}")

**Question:** How many parameters do we need to train?

**Answer:** The final linear layer has 1024 input features and 10 output classes, so:
- Weights: 1024 × 10 = 10,240
- Bias: 10
- **Total: 10,250 parameters**

This is very small compared to the full model (~2.5M parameters)!

## Training the new layer

### Creating loss function and optimizer

**TODO 5:** Create the loss function and optimizer.

Hints:
- Use CrossEntropyLoss for multi-class classification
- Use SGD optimizer with learning_rate=0.01 and momentum=0.9
- Only pass the parameters of the new layer to the optimizer: `model.classifier[3].parameters()`

Check the doc for:
- [nn.CrossEntropyLoss](https://docs.pytorch.org/docs/stable/generated/torch.nn.modules.loss.CrossEntropyLoss.html)
- [torch.optim.SGD](https://docs.pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [ ]:
# TODO 5: Define loss and optimizer
criterion = None #TODO
optimizer = None #TODO

### Training loop

Now let's train the model. The training function performs:
1. Forward pass: compute predictions
2. Loss computation: compare predictions with true labels
3. Backward pass: compute gradients
4. Optimizer step: update weights

In [ ]:
from tqdm import tqdm 

def train_model(model, dataloader, size, epochs=1, optimizer=None):
    """Train the model for given number of epochs."""
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, classes in tqdm(dataloader):
            inputs, classes = inputs.to(device), classes.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, classes)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Statistics
            _, preds = torch.max(outputs.data, 1)
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
        
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.data.item() / size
        print(f'Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

**TODO 6:** Train the model for 3 epochs.

Note: This should run on CPU in a reasonable time, but still need a few minutes per epoch.

In [ ]:
%%time
# TODO 6: Train the model


## Evaluation

Now let's test our model on the test set.

In [ ]:
def test_model(model, dataloader, size):
    """Evaluate the model on test data."""
    model.eval()
    predictions = np.zeros(size)
    all_classes = np.zeros(size)
    all_proba = np.zeros((size, 10))
    
    i = 0
    running_loss = 0.0
    running_corrects = 0
    
    with torch.no_grad():
        for inputs, classes in dataloader:
            inputs = inputs.to(device)
            classes = classes.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, classes)
            
            _, preds = torch.max(outputs.data, 1)
            
            # Statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == classes.data)
            
            predictions[i:i+len(classes)] = preds.to('cpu').numpy()
            all_classes[i:i+len(classes)] = classes.to('cpu').numpy()
            all_proba[i:i+len(classes), :] = nn.functional.softmax(outputs, dim=1).to('cpu').numpy()
            i += len(classes)
    
    epoch_loss = running_loss / size
    epoch_acc = running_corrects.data.item() / size
    print(f'Test Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    
    return predictions, all_proba, all_classes

In [ ]:
%%time
predictions, all_proba, all_classes = test_model(model, test_loader, size=dset_sizes['test'])

## Model analysis

Let's analyze the model's predictions to understand its behavior.

### 1. Correct predictions

First, let's look at some correct predictions:

In [ ]:
# Overall accuracy
correct = [] #TODO
accuracy = len(correct) / dset_sizes['test']
print(f"Test accuracy: {accuracy:.4f} ({len(correct)}/{dset_sizes['test']} correct)")

In [ ]:
# Visualize some correct predictions
from numpy.random import permutation

n_view = 16
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for idx, x in enumerate(permutation(correct)[:n_view]):
    img, label = test_dataset[x]
    img_np = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_np = np.clip(std * img_np + mean, 0, 1)
    
    axes[idx].imshow(img_np)
    axes[idx].set_title(f'True: {int(all_classes[x])}\nPred: {int(predictions[x])}\nConf: {all_proba[x, int(predictions[x])]:.2f}')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 2. Incorrect predictions

Let's examine where the model fails:

In [ ]:
# Visualize incorrect predictions
incorrect = [] #TODO
print(f"Number of incorrect predictions: {len(incorrect)}")

if len(incorrect) > 0:
    n_view = min(16, len(incorrect))
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.ravel()
    
    for idx, x in enumerate(permutation(incorrect)[:n_view]):
        img, label = test_dataset[x]
        img_np = img.numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img_np = np.clip(std * img_np + mean, 0, 1)
        
        axes[idx].imshow(img_np)
        axes[idx].set_title(f'True: {int(all_classes[x])}\nPred: {int(predictions[x])}\nConf: {all_proba[x, int(predictions[x])]:.2f}', 
                           color='red')
        axes[idx].axis('off')
    
    # Hide unused subplots
    for idx in range(n_view, 16):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

### 3. Most confident predictions

Let's look at predictions where the model is most confident:

In [ ]:
# Most confident predictions (highest probability)
confidences = [] #TODO
most_confident_idx = [] #TODO

fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for idx, x in enumerate(most_confident_idx):
    img, label = test_dataset[x]
    img_np = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_np = np.clip(std * img_np + mean, 0, 1)
    
    axes[idx].imshow(img_np)
    is_correct = predictions[x] == all_classes[x]
    color = 'green' if is_correct else 'red'
    axes[idx].set_title(f'True: {int(all_classes[x])}\nPred: {int(predictions[x])}\nConf: {confidences[x]:.4f}', 
                       color=color)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 4. Most uncertain predictions

Finally, let's examine cases where the model is uncertain:

In [ ]:
# Most uncertain predictions (probability closest to uniform)
uncertainty = [] #TODO  # Lower confidence = higher uncertainty
most_uncertain_idx = [] #TODO

fig, axes = plt.subplots(4, 4, figsize=(12, 12))
axes = axes.ravel()

for idx, x in enumerate(most_uncertain_idx):
    img, label = test_dataset[x]
    img_np = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_np = np.clip(std * img_np + mean, 0, 1)
    
    axes[idx].imshow(img_np)
    is_correct = predictions[x] == all_classes[x]
    color = 'green' if is_correct else 'red'
    
    # Show top 2 predictions
    top2_idx = np.argsort(-all_proba[x])[:2]
    top2_prob = all_proba[x][top2_idx]
    
    axes[idx].set_title(f'True: {int(all_classes[x])}\nTop: {top2_idx[0]}({top2_prob[0]:.2f}), {top2_idx[1]}({top2_prob[1]:.2f})', 
                       color=color, fontsize=9)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

### 5. Confusion matrix

A confusion matrix helps us understand which digits are confused with each other:

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(all_classes, predictions)

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=range(10), yticklabels=range(10))
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.title('Confusion Matrix')
plt.show()

### 6. Per-class accuracy

Let's see how well the model performs on each digit:

In [ ]:
# Per-class accuracy
for digit in range(10):
    digit_mask = all_classes == digit
    digit_correct = np.sum((predictions == all_classes) & digit_mask)
    digit_total = np.sum(digit_mask)
    digit_acc = digit_correct / digit_total
    print(f"Digit {digit}: {digit_acc:.4f} ({digit_correct}/{digit_total})")

## Conclusion

In this practical, you have:

1. **Learned transfer learning**: You used a model pre-trained on ImageNet (natural images) and adapted it to MNIST (handwritten digits)
2. **Understood domain adaptation**: MNIST images are very different from ImageNet (grayscale, simple shapes vs. color, complex objects), yet transfer learning still works!
3. **Trained efficiently**: By freezing pre-trained weights and training only the final layer, you trained the model in just a few minutes
4. **Achieved good performance**: With only ~12,800 trainable parameters, the model likely achieved >90% accuracy

### Extension ideas:

Try modifying the code to:
1. **Fine-tune more layers**: Unfreeze some earlier layers and train with a smaller learning rate
2. **Add data augmentation**: Apply random rotations, translations to improve robustness
3. **Compare architectures**: Try other small models used for classification
4. **Use a different dataset**: Try Fashion-MNIST or CIFAR-10